In [1]:
import vk                                                                       
import time                                                                     
import pickle                                                                   
import numpy as np 
import collections
import itertools
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
from collections import defaultdict                                             
                                                                                
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)  

In [2]:
all_members = {}
all_members.update(load_obj('./Anya_persons'))
print(len(all_members))
all_members.update(load_obj('./Dasha_persons'))
print(len(all_members))
all_members.update(load_obj('./Maks_persons'))

2337
4792


In [4]:
users_with_no_inf = []
groups_l = set()
for pers in all_members:
    try:
        groups_l.update(all_members[pers]['groups']) 
    except Exception: users_with_no_inf.append(pers)
groups_l = list(groups_l)

In [5]:
groups = {idx:group for idx, group in enumerate(groups_l)}
reverse_groups = {group:idx for idx, group in enumerate(groups_l)}


In [6]:
print(max(groups.keys()))
print(max(reverse_groups.keys()))

324701
189381412


In [7]:
for pers in users_with_no_inf:
    del all_members[pers]

In [8]:
users_l = list(all_members.keys())
users = {idx:user for idx, user in enumerate(users_l)}
reverse_users = {user:idx for idx, user in enumerate(users_l)}

In [9]:
data = []
row = []
col =[]
for idx, (user, val) in enumerate(all_members.items()):
    for group in val['groups'][0:int(0.33*len(val['groups']))]:
        row.append(idx)
        data.append(3)
        col.append(reverse_groups[group])
    for group in val['groups'][int(0.33*len(val['groups'])):int(0.66*len(val['groups']))]:
        row.append(idx)
        data.append(2)
        col.append(reverse_groups[group])
    for group in val['groups'][int(0.66*len(val['groups'])):]:
        row.append(idx)
        data.append(1)
        col.append(reverse_groups[group])

In [10]:
csr = csr_matrix((np.array(data), (np.array(row),np.array(col))))

In [11]:
csr.shape


(7132, 324702)

In [12]:
def recs(idx, csr_a):
    # считаем косинус между всеми пользователями
    metrics = cosine_similarity(csr_a.getrow(idx).toarray().reshape(1,-1), csr_a).reshape(-1, 1)
    # домножаем оценки пользовтеля на коэффициент похожести

    csr_a = csr_a.multiply( metrics)
    total_rate = np.sum(csr_a, axis=0)
    return total_rate

In [13]:
dasha = reverse_users[9183085]
dasha

1822

In [14]:

t = recs(dasha, csr)

In [15]:
a = t.tolist()

In [16]:
a = a[0]

In [17]:
a = [(j,i) for i , j in enumerate(a) ]

In [18]:
a.sort(key=lambda i: i[0], reverse=True)

In [19]:
#reverse_groups = {val:key for key, val in groups.items()}
i = a[:10]
print([groups[idx] for score, idx in i])

[1441, 54295855, 29534144, 29559271, 49866476, 31976785, 65652356, 63731512, 37119411, 41437811]


In [29]:
def precision_for_one(csr, user_l):
    user = reverse_users[user_l]
    t = recs(user, csr)
    a = t.tolist()
    a = a[0]
    a = [(j,i) for i , j in enumerate(a)]
    a.sort(key=lambda i: i[0], reverse=True)
    i = a[:20]
    recomend = set([groups[idx] for score, idx in i])
    groups_of_user = set(all_members[user_l]['groups'])
    TP = len(recomend&groups_of_user)
    return TP/20.

In [30]:
def recall_for_one(csr, user_l):
    user = reverse_users[user_l]
    t = recs(user, csr)
    a = t.tolist()
    a = a[0]
    a = [(j,i) for i , j in enumerate(a)]
    a.sort(key=lambda i: i[0], reverse=True)
    i = a[:20]
    recomend = set([groups[idx] for score, idx in i])
    groups_of_user = set(all_members[user_l]['groups'])
    TP = len(recomend&groups_of_user)
    return TP/len(groups_of_user)

In [31]:
import random
users_for_test = random.sample(users_l, int(0.33*len(users_l)))

In [32]:
list_of_prec = []
list_of_recall = []
for i in users_for_test:
    list_of_prec.append(precision_for_one(csr, i))
    list_of_recall.append(recall_for_one(csr, i))

In [33]:
sum(list_of_prec)/len(list_of_prec)

0.32407564810879674

In [34]:
sum(list_of_recall)/len(list_of_recall)

0.1342450059333986